# 하이퍼파라미터 튜닝 Job 결과 분석하기
## [(원본)](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/hyperparameter_tuning/analyze_results)


튜닝 Job이 끝나면 (혹은 Job이 계속 실행되는 동안에도) 이 노트북을 사용하여 그 결과를 분석해 각 하이퍼파라미터가 모델의 품질에 얼마나 영향을 미쳤는지 이해할 수 있습니다.


---
## 환경 설정하기
분석을 시작하기 위해 하이퍼파라미터 튜닝 Job의 이름을 선택해야 합니다. 

In [21]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

tuning_job_name = 'YOUR-HYPERPARAMETER-TUNING-JOB-NAME'


## 하이퍼파라미터 튜닝 Job 진행 상황 추적하기
튜닝 Job을 실행한 후에 describe_tuning_job API를 호출하여 진행상황을 확인할 수 있습니다. describe_tuning_job의 출력 형식은 JSON Object이고 이것은 튜닝 Job의 현재 상태에 대한 정보를 포함합니다. list_training_jobs_for_tuning_job 를 호출해서 튜닝 Job이 시작한 훈련 Job들의 자세한 목록을 볼 수 있습니다. 

In [15]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

9 training jobs have completed


In [16]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2019, 11, 25, 14, 59, 28, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'loss',
                                                 'Value': 0.09912024438381195},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2019, 11, 25, 15, 5, 13, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:415373942856:training-job/sagemaker-tensorflow-191125-1447-007-97d139b7',
 'TrainingJobName': 'sagemaker-tensorflow-191125-1447-007-97d139b7',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2019, 11, 25, 15, 2, 4, tzinfo=tzlocal()),
 'TunedHyperParameters': {'learning_rate': '0.017117079614152788'}}



## DataFrame으로 모든 결과를 가져오기

하이퍼파라미터와 모든 훈련 Job의 목표 매트릭들을 리스팅할 수 있고 최상의 목표 매트릭을 갖는 훈련 Job을 선택할 수 있습니다. 

In [17]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 9
{'lowest': 0.09912024438381195, 'highest': 2.302058696746826}


,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,learning_rate
2,0.099120,189.0,2019-11-25 15:05:13+00:00,sagemaker-tensorflow-191125-1447-007-97d139b7,Completed,2019-11-25 15:02:04+00:00,0.017117
7,0.100829,197.0,2019-11-25 14:52:54+00:00,sagemaker-tensorflow-191125-1447-002-efd1d965,Completed,2019-11-25 14:49:37+00:00,0.017146
0,0.114712,217.0,2019-11-25 15:05:13+00:00,sagemaker-tensorflow-191125-1447-009-daf5226e,Completed,2019-11-25 15:01:36+00:00,0.017163
8,2.301762,193.0,2019-11-25 14:52:58+00:00,sagemaker-tensorflow-191125-1447-001-a599ebf1,Completed,2019-11-25 14:49:45+00:00,0.013991
6,2.301783,179.0,2019-11-25 14:52:48+00:00,sagemaker-tensorflow-191125-1447-003-0117fe61,Completed,2019-11-25 14:49:49+00:00,0.014113
1,2.301999,190.0,2019-11-25 15:04:46+00:00,sagemaker-tensorflow-191125-1447-008-898eb7ff,Completed,2019-11-25 15:01:36+00:00,0.017141
4,2.302008,193.0,2019-11-25 14:58:35+00:00,sagemaker-tensorflow-191125-1447-005-4123fb8c,Completed,2019-11-25 14:55:22+00:00,0.017370
3,2.302024,188.0,2019-11-25 14:58:32+00:00,sagemaker-tensorflow-191125-1447-006-e2c04f3f,Completed,2019-11-25 14:55:24+00:00,0.017464
5,2.302059,196.0,2019-11-25 14:59:01+00:00,sagemaker-tensorflow-191125-1447-004-3a3f33fa,Completed,2019-11-25 14:55:45+00:00,0.017904



## 튜닝 Job 결과와 time 관계 확인
다음으로 튜닝 Job이 진행되고 시간에 지남에 따라 목표 매트릭이 어떻게 변했는지 보여줍니다. Bayesian strategy에 경우, 보다 나은 결과를 향한 일반적인 추세를 기대하겠지만 알고리즘은 파라미터의 새로운 영역의 _exploration_ 과 알려진 양호한 영역의 _exploitation_ 의 균형을 유지해야 하기 때문에 이 프로그레스는 안정적이지 않을 것입니다. 이를 통해 검색 공간의 복잡성에 훈련 Job의 수가 충분한지 여부를 알 수 있습니다. 


In [19]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...


## 목표 매트릭과 개별 하이퍼파라미터의 상관관계 분석하기

이제 튜닝 Job을 마쳤으므로 목표 매트릭과 튜닝을 선택한 개별 하이퍼파라미터의 상관관계를 알고 싶을 것입니다. 그런 통찰력을 가지면 특정 하이퍼파라미터의 검색범위를 조정하고 다른 튜닝 Job을 시작하는 것이 적합한지 여부를 결정하는데 도움이 될 것입니다. 예를 들어 목표 매트릭과 숫자 하이퍼파라미터 사이에 긍정적인 경향이 보이면, 다음 튜닝 Job에서 더 높은 하이퍼파라미터의 튜닝 범위를 설정할 것 입니다. 

다음 셀은 개별 하이퍼파라미터에 대한 그래프를 표시하여 목표 매트릭간의 상관관계를 보여줍니다. 

In [20]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))